# 轮廓一定是连续的封闭图像

In [ ]:
# -*- coding: utf-8 -*-
import cv2

# ============================================
# 1️⃣ 读取图像
# ============================================
image_np = cv2.imread("../image/work4.png")

# 转换为灰度图
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# ============================================
# 2️⃣ 二值化处理
# ============================================
# cv2.threshold(src, thresh, maxval, type)
#   - src：输入灰度图像
#   - thresh：阈值（127表示灰度分界）
#   - maxval：满足条件的像素被赋予的最大值（通常255）
#   - type：阈值化类型
#       cv2.THRESH_BINARY      →  大于阈值为255，否则为0
#       cv2.THRESH_BINARY_INV  →  大于阈值为0，否则为255
#       cv2.THRESH_TRUNC       →  大于阈值设为阈值本身
#       cv2.THRESH_TOZERO      →  小于阈值设为0
#       cv2.THRESH_TOZERO_INV  →  大于阈值设为0
_, image_thresh = cv2.threshold(image_gray, 127, 255, cv2.THRESH_BINARY_INV)

# ============================================
# 3️⃣ 查找轮廓
# ============================================
# cv2.findContours(image, mode, method[, offset])
# --------------------------------------------
# 参数：
#   image：输入图像（必须是二值图，非零像素视为前景）
#   mode ：轮廓检索模式（决定是否建立层级关系）
#       cv2.RETR_EXTERNAL ：仅检测最外层轮廓，忽略内嵌的轮廓
#       cv2.RETR_LIST     ：检测所有轮廓，但不建立层级关系（全部平级）
#       cv2.RETR_CCOMP    ：仅建立两层结构（外层与内层）
#       cv2.RETR_TREE     ：建立完整的轮廓树（含父子、兄弟关系）
#
#   method：轮廓近似方式（决定存储多少边界点）
#       cv2.CHAIN_APPROX_NONE     ：保留轮廓上的所有点（最精确）
#       cv2.CHAIN_APPROX_SIMPLE   ：压缩冗余点，仅保留拐点（默认推荐）
#       cv2.CHAIN_APPROX_TC89_L1  ：Teh-Chin近似算法（更平滑）
#       cv2.CHAIN_APPROX_TC89_KCOS：Teh-Chin近似算法（另一种改进形式）
#
#   offset：（可选）偏移量，若设定会平移轮廓点的坐标
#
# 返回：
#   contours ：Python列表，每个元素是一条轮廓（np.ndarray类型，形状为(N, 1, 2)）
#               每个点是像素坐标 [x, y]
#   hierarchy：层级关系数组（shape=(1, N, 4)）
#               每个轮廓对应 [next, prev, child, parent]
#               含义如下：
#                   next   → 同级下一个轮廓索引（右兄弟），无为 -1
#                   prev   → 同级上一个轮廓索引（左兄弟），无为 -1
#                   child  → 第一个子轮廓索引（内部轮廓），无为 -1
#                   parent → 外层父轮廓索引，无为 -1
contours, hierarchy = cv2.findContours(
    image_thresh,
    cv2.RETR_EXTERNAL,     # 只取最外层轮廓（可以改为 LIST / TREE / CCOMP）
    cv2.CHAIN_APPROX_SIMPLE # 压缩点集表示
)

# ============================================
# 4️⃣ 绘制轮廓
# ============================================
# cv2.drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]])
# 参数说明：
#   image：绘制目标图像
#   contours：findContours()返回的轮廓列表
#   contourIdx：
#       -1 表示绘制所有轮廓
#       >=0 表示绘制指定编号的轮廓
#   color：BGR颜色值 (255, 255, 255)=白色
#   thickness：线宽（>0为线条，-1表示填充轮廓内部）
#   lineType：线条类型（默认8连接线）
#   hierarchy/maxLevel：用于控制绘制层级（通常默认）
#   offset：绘制时的坐标偏移
cv2.drawContours(image_np, contours, -1, (255, 255, 255), 2)

# ============================================
# 5️⃣ 显示结果
# ============================================
cv2.imshow("Binary Image", image_thresh)
cv2.imshow("Contours", image_np)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ============================================
# 📘 扩展操作（常见轮廓属性计算）
# ============================================
# 输出轮廓数量
print(f"检测到 {len(contours)} 个轮廓")

# 打印每个轮廓的面积与周长
for i, c in enumerate(contours):
    area = cv2.contourArea(c)        # 计算轮廓面积
    perimeter = cv2.arcLength(c, True)  # 计算轮廓周长（True表示闭合）
    print(f"轮廓 {i}: 面积={area:.2f}, 周长={perimeter:.2f}")

# 查看层级结构
print("层级信息 hierarchy:\n", hierarchy)


-1